# 初始代码（4o）

In [ ]:
import numpy as np
from scipy.integrate import dblquad
from scipy.optimize import minimize_scalar

# 定义新的损失函数 L(v1, v2, a)，考虑超车概率和不同车道情况
def integrand_total_loss(v1, v2, a):
    if v1 <= v2:
        return 0  # 不构成超车，不考虑
    elif v1 < a and v2 < a:
        # 两车在慢车道：损失是 v2^2，乘以超车概率 (v1 - v2)
        return (v1 - v2)/(v1*v2) * v2**2
    elif v1 > a and v2 > a:
        # 两车在快车道：损失是 (v2 - a)^2，乘以超车概率 (v1 - v2)
        return (v1 - v2)/(v1*v2) * (v2 - a)**2
    else:
        # 一快一慢，分在不同车道，不构成阻碍，损失为0
        return 0

# 对每一个 a ∈ [1,2] 计算期望损失的积分
def expected_loss(a):
    # 积分区域是 v2 in [1,2], v1 in [v2, 2] （确保 v1 > v2）
    result, _ = dblquad(
        lambda v1, v2: integrand_total_loss(v1, v2, a),
        1, 2,  # v2 下限和上限
        lambda v2: v2,  # v1 下限
        lambda v2: 2    # v1 上限
    )
    return result
# 使用带界优化在 a ∈ [1, 2] 上最小化期望损失
res = minimize_scalar(expected_loss, bounds=(1, 2), method='bounded')
print(res)
optimal_a = round(res.x, 12)
optimal_a

 message: Solution found.
 success: True
  status: 0
     fun: 0.0030155855822074904
       x: 1.1771860419694444
     nit: 12
    nfev: 12


np.float64(1.177186041969)

# 尝试精度优化

In [55]:
import numpy as np
from scipy.integrate import dblquad
from scipy.optimize import minimize_scalar
from scipy.integrate import quad

def part_slow(a):
    # v2∈[1,a]  v1∈[v2,a]
    return quad(
        lambda v2: quad(
            lambda v1: (v1-v2)/(v1*v2)*v2**2,
            v2, a, epsabs=1e-12, epsrel=1e-12
        )[0],
        1, a, epsabs=1e-12, epsrel=1e-12
    )[0]

def part_fast(a):
    # v2∈[a,2]  v1∈[v2,2]
    return quad(
        lambda v2: quad(
            lambda v1: (v1-v2)/(v1*v2)*(v2-a)**2,
            v2, 2, epsabs=1e-12, epsrel=1e-12
        )[0],
        a, 2, epsabs=1e-12, epsrel=1e-12
    )[0]

def expected_loss(a):
    return part_slow(a) + part_fast(a)

res = minimize_scalar(expected_loss,
    bounds=(1, 2), 
    method='bounded',
    options={'xatol':1e-12})
print(res)
optimal_a = round(res.x, 12)
optimal_a

 message: Solution found.
 success: True
  status: 0
     fun: 0.003015586417843473
       x: 1.1771414248396577
     nit: 11
    nfev: 11


np.float64(1.17714142484)

### 误差检验
  epsabs=1e-12使得quad只保证对一次一维积分误差不超过1e-12，两层quad使得误差累积（1e-10），slow & fast part会继续累积误差
  a只改动10**（-9），真实损失函数变化＜误差抖动，优化器失效 

In [57]:
val, err = part_slow(1.18), part_fast(1.18)
print("估算值:", val + err)
print("quad 报告误差:", err)

估算值: 0.0030165216954305693
quad 报告误差: 0.0021210422029061043


## 进一步优化

In [ ]:
import mpmath as mp

mp.dps = 40        # 工作 40 位小数
def part_slow_mp(a):
    f1 = lambda v1, v2: (v1-v2)/(v1*v2)*v2**2
    return mp.quad(lambda v2: mp.quad(lambda v1: f1(v1, v2), [v2, a]), [1, a])

def part_fast_mp(a):
    f2 = lambda v1, v2, a: (v1-v2)/(v1*v2)*(v2-a)**2
    return mp.quad(lambda v2: mp.quad(lambda v1: f2(v1, v2, a), [v2, 2]), [a, 2])

def expected_loss_mp(a):
    return part_slow_mp(a) + part_fast_mp(a)

d_expected_loss = lambda a: mp.diff(expected_loss_mp, a)
root = mp.findroot(d_expected_loss, 1.18)
print(root)        

1.1771414168155060174
